In [10]:
import json
import os
import scipy.ndimage
import numpy as np
import matplotlib.path as mpltPath
import pdb

#base_path = '/home/ubuntu/data/'
base_path = '/home/ubuntu/data'
slide_ids = [int(x) for x in os.listdir(base_path+"/patches/") if x.isdigit()]

#f = open("/mys3bucket/Annotations/annotations.txt", encoding="utf-8")

f = open("/home/ubuntu/data/Annotations/annotations.txt", encoding="utf-8")
data = json.loads(f.read())
f.close()

slide_based_regions = {}
for annotation in data:
    slide = annotation['slideId']
    shape = annotation['shape']
    if slide in slide_ids and len(shape) > 1:
        if slide not in slide_based_regions:
            slide_based_regions[slide] = []
            slide_based_regions[slide].append(shape)
        else:
            slide_based_regions[slide].append(shape)

def isInside(point, polygon):
    path = mpltPath.Path(polygon)
    return path.contains_point(point)

X_train = []
Y_train = []
count = 0 
for slide_id in slide_based_regions.keys():
    
    
        
    slide_csv_path = base_path + "/patches/" + str(slide_id) + "/" + str(slide_id) + ".csv"
    patch_regions = {}

    f = open(slide_csv_path, "r")
    lines = f.readlines()
    f.close()
    
    for line in lines:
        patch_id = int((line.strip().split(',')[0]).split('_')[1])
        x, y = int(line.strip().split(',')[1]), int(line.strip().split(',')[2])
        center_x, center_y = x + 128, y + 128
        patch_regions[patch_id] = (center_x, center_y)
        
    print("-----------------------")
    
    shapes = slide_based_regions[slide_id]
    for patch_id in patch_regions.keys():
        center_point = patch_regions[patch_id]
        flag = 0
        for shape in shapes:
            if isInside(center_point, shape):
                flag = 1
                break

        Y_train.append(flag)
        patch_path = base_path + "/patches/" + str(slide_id) + "/patch_" + str(patch_id) + ".jpg"
        image = scipy.ndimage.imread(patch_path)
        X_train.append(image)
    count+=1
    if count == 10:
        break

X_train = np.array(X_train)
Y_train = np.array(Y_train)
np.savez("training_data", X=X_train, Y=Y_train)
print("Done!")

-----------------------


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
-----------------------
Done!
